Pada proyek kali ini, Anda diminta untuk mengolah data pendaftar hackathon yang diselenggarakan oleh DQLab bernama DQThon.

Dataset ini terdiri dari 5000 baris data (5000 pendaftar) dengan format CSV (Comma-separated values) dan memiliki beberapa kolom diantaranya:
<ol>
    <li>participant_id: ID dari peserta/partisipan hackathon. Kolom ini bersifat unique sehingga antar peserta pasti memiliki ID yang berbeda</li>
    <li>first_name: nama depan peserta</li>
    <li>last_name: nama belakang peserta</li>
    <li>birth_date: tanggal lahir peserta</li>
    <li>address: alamat tempat tinggal peserta</li>
    <li>phone_number: nomor hp/telepon peserta</li>
    <li>country: negara asal peserta</li>
    <li>institute: institusi peserta saat ini, bisa berupa nama perusahaan maupun nama universitas</li>
    <li>occupation: pekerjaan peserta saat ini</li>
    <li>register_time: waktu peserta melakukan pendaftaran hackathon dalam second</li>
</ol>
Namun pada proyek ini nantinya Anda diminta untuk menghasilkan beberapa kolom dengan memanfaatkan kolom-kolom yang ada, sehingga akhir dari proyek ini berupa hasil transformasi data dengan beberapa kolom baru selain dari 10 kolom diatas.

Sebagai pemanasan dalam proyek ini, Anda dipersilakan untuk membuka isi datasetnya dan melihat values-nya.

<h3>Extract</h3>
Extract merupakan proses meng-ekstraksi data dari sumber, sumber data ini bisa berupa relational data (SQL) atau tabel, nonrelational (NoSQL) maupun yang lainnya.

Tugas Anda adalah baca terlebih dahulu dataset ini sebagai CSV agar nantinya bisa diolah. Gunakan live code editor untuk menampilkan dataset.

File tersebut bisa diakses di path dqthon-participants.csv.

In [1]:
import pandas as pd

df_participant = pd.read_csv('https://storage.googleapis.com/dqlab-dataset/dqthon-participants.csv')
df_participant.head()
df_participant.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5000 entries, 0 to 4999
Data columns (total 10 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   participant_id  5000 non-null   object 
 1   first_name      5000 non-null   object 
 2   last_name       5000 non-null   object 
 3   birth_date      5000 non-null   object 
 4   address         5000 non-null   object 
 5   phone_number    5000 non-null   object 
 6   country         5000 non-null   object 
 7   institute       5000 non-null   object 
 8   occupation      5000 non-null   object 
 9   register_time   5000 non-null   float64
dtypes: float64(1), object(9)
memory usage: 390.8+ KB


<h3>Transform Bagian I - Kode Pos</h3>
Ada permintaan datang dari tim logistik bahwa mereka membutuhkan kode pos dari peserta agar pengiriman piala lebih mudah dan cepat sampai. Maka dari itu buatlah kolom baru bernama postal_code yang memuat informasi mengenai kode pos yang diambil dari alamat peserta (kolom address).

Diketahui bahwa kode pos berada di paling akhir dari alamat tersebut.

Note:
Jika regex yang dimasukkan tidak bisa menangkap pattern dari value kolom address maka akan menghasilkan NaN.

In [2]:
# Raw address column
df_participant['address'].head()

0             Gg. Monginsidi No. 08\nMedan, Aceh 80734
1       Gg. Rajawali Timur No. 7\nPrabumulih, MA 09434
2               Jalan Kebonjati No. 0\nAmbon, SS 57739
3      Jl. Yos Sudarso No. 109\nLubuklinggau, SR 76156
4    Jalan Gardujati No. 53\nKediri, Sulawesi Tenga...
Name: address, dtype: object

In [3]:
# Create Postal Code Column
df_participant['postal_code'] = df_participant['address'].str.extract(
    r'(\d+)$')  #Masukkan regex Anda didalam fungsi extract
df_participant['postal_code']

0       80734
1       09434
2       57739
3       76156
4       70074
        ...  
4995    50268
4996    57434
4997    95312
4998    80657
4999    41134
Name: postal_code, Length: 5000, dtype: object

<h3>Transform Bagian II - Kota</h3>
Selain kode pos, mereka juga membutuhkan kota dari peserta.

Untuk menyediakan informasi tersebut, buatlah kolom baru bernama city yang didapat dari kolom address. Diasumsikan bahwa kota merupakan sekumpulan karakter yang terdapat setelah nomor jalan diikuti dengan \n (newline character) atau dalam bahasa lainnya yaitu enter.

In [4]:
# Create City Column
df_participant['city'] = df_participant['address'].str.extract(r'(?<=\n)(\w.+)(?=,)')
df_participant['city'].head()

0           Medan
1      Prabumulih
2           Ambon
3    Lubuklinggau
4          Kediri
Name: city, dtype: object

<h3>Transform Bagian III - Github</h3>
Salah satu parameter untuk mengetahui proyek apa saja yang pernah dikerjakan oleh peserta yaitu dari git repository mereka.

Pada kasus ini kita menggunakan profil github sebagai parameternya. Tugas Anda yaitu membuat kolom baru bernama github_profile yang merupakan link profil github dari peserta.

Diketahui bahwa profil github mereka merupakan gabungan dari first_name dan last_name yang sudah di-lowercase. 

In [5]:
# Create github profile column
df_participant['github_profile'] = 'https://github.com/' + df_participant[
    'first_name'].str.lower() + df_participant['last_name'].str.lower()
df_participant['github_profile'].head()

0      https://github.com/citranurdiyanti
1         https://github.com/arissetiawan
2    https://github.com/corneliahandayani
3          https://github.com/solehrajasa
4           https://github.com/viviastuti
Name: github_profile, dtype: object

<h3>Transform Bagian IV - Nomor Handphone</h3>
Jika kita lihat kembali, ternyata nomor handphone yang ada pada data csv kita memiliki format yang berbeda-beda. Maka dari itu, kita perlu untuk melakukan cleansing pada data nomor handphone agar memiliki format yang sama. Anda sebagai Data Engineer diberi privilege untuk menentukan format nomor handphone yang benar. Pada kasus ini mari kita samakan formatnya dengan aturan:
<ol>
    <li>Jika awalan nomor HP berupa angka 62 atau +62 yang merupakan kode telepon Indonesia, maka diterjemahkan ke 0.</li>
    <li>Tidak ada tanda baca seperti kurung buka, kurung tutup, strip⟶ ()-</li>
    <li>Tidak ada spasi pada nomor HP nama kolom untuk menyimpan hasil cleansing pada nomor HP yaitu cleaned_phone_number</li>
</ol>

In [6]:
# Raw Phone Number
df_participant['phone_number'].head()

0       (0151) 081 2706
1    +62 (036) 461 7027
2          089 833 6695
3    +62 (418) 329-4756
4            0812511835
Name: phone_number, dtype: object

In [7]:
# Transform Phone Number to Cleaned phone number
df_participant['cleaned_phone_number'] = df_participant[
    'phone_number'].str.replace(r'^(\+62|62)', '0')
df_participant['cleaned_phone_number'] = df_participant[
    'cleaned_phone_number'].str.replace(r'[()-]', '')
df_participant['cleaned_phone_number'] = df_participant[
    'cleaned_phone_number'].str.replace(r'\s+', '')
df_participant['cleaned_phone_number'].head()

C:\Users\User\AppData\Local\Temp/ipykernel_7916/4081023883.py:2: FutureWarning: The default value of regex will change from True to False in a future version.
  df_participant['cleaned_phone_number'] = df_participant[
C:\Users\User\AppData\Local\Temp/ipykernel_7916/4081023883.py:4: FutureWarning: The default value of regex will change from True to False in a future version.
  df_participant['cleaned_phone_number'] = df_participant[
C:\Users\User\AppData\Local\Temp/ipykernel_7916/4081023883.py:6: FutureWarning: The default value of regex will change from True to False in a future version.
  df_participant['cleaned_phone_number'] = df_participant[


0    01510812706
1    00364617027
2     0898336695
3    04183294756
4     0812511835
Name: cleaned_phone_number, dtype: object

<h3>Transform Bagian V - Nama Tim</h3>
Dataset saat ini belum memuat nama tim, dan rupanya dari tim Data Analyst membutuhkan informasi terkait nama tim dari masing-masing peserta.

Diketahui bahwa nama tim merupakan gabungan nilai dari kolom first_name, last_name, country dan institute.

Tugas Anda yakni buatlah kolom baru dengan nama team_name yang memuat informasi nama tim dari peserta.

In [8]:
def func(col):
    abbrev_name = "%s%s" % (col['first_name'][0], col['last_name'][0])
    country = col['country']
    abbrev_institute = '%s' % (''.join(
        list(map(lambda word: word[0], col['institute'].split()))))
    return "%s-%s-%s" % (abbrev_name, country, abbrev_institute)


df_participant['team_name'] = df_participant.apply(func, axis=1)
df_participant['team_name'].head()

0       CN-Georgia-UPM
1    AS-Korea Utara-UD
2        CH-Komoro-UHP
3       SR-Eritrea-PTY
4      VA-Aljazair-PHR
Name: team_name, dtype: object

<h3>Transform Bagian VI - Email</h3>
Setelah dilihat kembali dari data peserta yang dimiliki, ternyata ada satu informasi yang penting namun belum tersedia, yaitu email.

Anda sebagai Data Engineer diminta untuk menyediakan informasi email dari peserta dengan aturan bahwa format email sebagai berikut:
<pre>
Format email:
xxyy@aa.bb.[ac/com].[cc]

Keterangan:
xx -> nama depan (first_name) dalam lowercase
yy -> nama belakang (last_name) dalam lowercase
aa -> nama institusi

Untuk nilai bb, dan cc mengikuti nilai dari aa. Aturannya:
- Jika institusi nya merupakan Universitas, maka
  bb -> gabungan dari huruf pertama pada setiap kata dari nama Universitas dalam lowercase
  Kemudian, diikuti dengan .ac yang menandakan akademi/institusi belajar dan diikuti dengan pattern cc
- Jika institusi bukan merupakan Universitas, maka
  bb -> gabungan dari huruf pertama pada setiap kata dari nama Universitas dalam lowercase
  Kemudian, diikuti dengan .com. Perlu diketahui bahwa pattern cc tidak berlaku pada kondisi ini

cc -> merupakan negara asal peserta, adapun aturannya:
- Jika banyaknya kata pada negara tersebut lebih dari 1 maka ambil singkatan dari negara tersebut dalam lowercase
- Namun, jika banyaknya kata hanya 1 maka ambil 3 huruf terdepan dari negara tersebut dalam lowercase

Contoh:
  Nama depan: Citra
  Nama belakang: Nurdiyanti
  Institusi: UD Prakasa Mandasari
  Negara: Georgia
  Maka,Email nya: citranurdiyanti@upm.geo
  -----------------------------------
  Nama depan: Aris
  Nama belakang: Setiawan
  Institusi: Universitas Diponegoro
  Negara: Korea Utara
  Maka, Email nya: arissetiawan@ud.ac.ku
</pre>

In [9]:
def func(col):
    first_name_lower = col['first_name'].lower()
    last_name_lower = col['last_name'].lower()
    institute = ''.join(list(map(lambda word: word[0], col['institute'].lower().split()))) #Singkatan dari nama perusahaan dalam lowercase
    if 'Universitas' in col['institute']:
        if len(col['country'].split()) > 1: #Kondisi untuk mengecek apakah jumlah kata dari country lebih dari 1
            country = ''.join(list(map(lambda word: word[0],col['country'].lower().split())))
        else:
            country = col['country'][:3].lower()
        return "%s%s@%s.ac.%s"%(first_name_lower,last_name_lower,institute,country)
    
    return "%s%s@%s.com"%(first_name_lower,last_name_lower,institute)

df_participant['email'] = df_participant.apply(func, axis=1)
df_participant['email'].head()

0      citranurdiyanti@upm.com
1        arissetiawan@ud.ac.ku
2    corneliahandayani@uhp.com
3          solehrajasa@pty.com
4           viviastuti@phr.com
Name: email, dtype: object

<h3>Transform Bagian VII - Tanggal Lahir</h3>
MySQL merupakan salah satu database yang sangat populer dan digunakan untuk menyimpan data berupa tabel, termasuk data hasil pengolahan yang sudah kita lakukan ini nantinya bisa dimasukkan ke MySQL.

Meskipun begitu, ada suatu aturan dari MySQL terkait format tanggal yang bisa mereka terima yaitu YYYY-MM-DD dengan keterangan:

YYYY: 4 digit yang menandakan tahun
MM: 2 digit yang menandakan bulan
DD: 2 digit yang menandakan tanggal
Contohnya yaitu: 2021-04-07

Jika kita lihat kembali pada kolom tanggal lahir terlihat bahwa nilainya belum sesuai dengan format DATE dari MySQL

(Lihat lebih detail di: https://www.mysqltutorial.org/mysql-date/).

Oleh karena itu, lakukanlah formatting terhadap kolom birth_date menjadi YYYY-MM-DD dan simpan di kolom yang sama.

In [10]:
df_participant['birth_date'].head()

0    05 Feb 1991
1    11 Jan 1993
2    31 Jul 1993
3    04 Nov 1991
4    22 Jan 2003
Name: birth_date, dtype: object

In [11]:
df_participant['birth_date'] = pd.to_datetime(df_participant['birth_date'], format='%d %b %Y')
df_participant['birth_date']

0      1991-02-05
1      1993-01-11
2      1993-07-31
3      1991-11-04
4      2003-01-22
          ...    
4995   2000-04-27
4996   1996-03-06
4997   2003-02-14
4998   1994-08-22
4999   2002-02-09
Name: birth_date, Length: 5000, dtype: datetime64[ns]

<h3>Transform Bagian VII - Tanggal Daftar Kompetisi</h3>
Selain punya aturan mengenai format DATE, MySQL juga memberi aturan pada data yang bertipe DATETIME yaitu YYYY-MM-DD HH:mm:ss dengan keterangan:

YYYY: 4 digit yang menandakan tahun
MM: 2 digit yang menandakan bulan
DD: 2 digit yang menandakan tanggal
HH: 2 digit yang menandakan jam
mm: 2 digit yang menandakan menit
ss: 2 digit yang menandakan detik
Contohnya yaitu: 2021-04-07 15:10:55

Karena data kita mengenai waktu registrasi peserta (register_time) belum sesuai format yang seharusnya.

Maka dari itu, tugas Anda yaitu untuk merubah register_time ke format DATETIME sesuai dengan aturan dari MySQL.

Simpanlah hasil tersebut ke kolom register_at.

In [12]:
df_participant['register_time']

0       1.617634e+09
1       1.617634e+09
2       1.617634e+09
3       1.617634e+09
4       1.617634e+09
            ...     
4995    1.617634e+09
4996    1.617634e+09
4997    1.617634e+09
4998    1.617634e+09
4999    1.617634e+09
Name: register_time, Length: 5000, dtype: float64

In [13]:
df_participant['register_at'] = pd.to_datetime(df_participant['register_time'], unit='s')
df_participant['register_at']

0      2021-04-05 14:47:26
1      2021-04-05 14:46:58
2      2021-04-05 14:47:15
3      2021-04-05 14:47:14
4      2021-04-05 14:46:50
               ...        
4995   2021-04-05 14:46:16
4996   2021-04-05 14:46:48
4997   2021-04-05 14:46:06
4998   2021-04-05 14:46:45
4999   2021-04-05 14:46:51
Name: register_at, Length: 5000, dtype: datetime64[ns]

<h3>Load</h3>
Pada bagian load ini, data yang sudah ditransformasi sedemikian rupa sehingga sesuai dengan kebutuhan tim analyst dimasukkan kembali ke database yaitu Data Warehouse (DWH). Biasanya, dilakukan pendefinisian skema database terlebih dahulu seperti:
<ol>
    <li>Nama kolom</li>
    <li>Tipe kolom</li>
    <li>Apakah primary key, unique key, index atau bukan</li>
    <li>Panjang kolomnya</li>
</ol>
Karena umumnya Data Warehouse merupakan database yang terstruktur sehingga mereka memerlukan skema sebelum datanya dimasukkan.

Pandas sudah menyediakan fungsi untuk memasukkan data ke database yaitu to_sql().

Detail dari fungsi tersebut bisa dilihat pada dokumentasi Pandas: https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.to_sql.html